<a href="https://colab.research.google.com/github/GavinHacker/ml_practice/blob/master/recbycontent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
!pip install pymysql

    100% |████████████████████████████████| 51kB 2.0MB/s 


In [0]:
import pandas as pd
import pymysql
import pymysql.cursors
from functools import reduce
import numpy as np
import pandas as pd
import uuid
import datetime
from pyfm import pylibfm
from sklearn.feature_extraction import DictVectorizer
np.set_printoptions(precision=3)
np.set_printoptions(suppress=True)

In [0]:
def get_connection():
    return pymysql.connect(host='rm-2zeqqm6994abi7b6dqo.mysql.rds.aliyuncs.com',
                               user='noone',
                               password='Huawei12#$',
                               db='recsys',
                               port=3306,
                               charset ='utf8',
                               use_unicode=True)

In [0]:
df = pd.read_sql_query("select * from movie", get_connection())

In [7]:
df.shape

(6629, 10)

In [0]:
df_exp = df.head(10)

In [90]:
df_exp

,id,name,ADD_TIME,type,actors,region,director,trait,rat,enable
0,10001432,颠倒的帕特玛,2018-02-07 20:46:08,剧情|冒险|科幻|爱情,藤井雪代|冈本信彦|大畑伸太郎|福松进纱|加藤将之,日本,吉浦康裕,青春,7.9,1
1,10047547,饥饿游戏3：嘲笑鸟(下),2018-02-07 20:27:08,奇幻|惊悚|爱情|战争|剧情|科幻|冒险|动作,詹妮弗·劳伦斯|乔什·哈切森|利亚姆·海姆斯沃斯|伍迪·哈里森|伊丽莎白·班克斯,美国,弗朗西斯·劳伦斯,青春,6.2,1
2,10342838,弯刀杀戮,2018-02-07 20:42:15,喜剧|犯罪|惊悚|动作,丹尼·特雷霍|米歇尔·罗德里格兹|索菲娅·维加拉|艾梅柏·希尔德|Lady Gaga,俄罗斯,罗伯特·罗德里格兹,搞笑,5.7,1
3,10355329,巴菲的奇妙命运,2018-02-07 20:44:01,剧情|喜剧|音乐|爱情,佩丽冉卡·曹帕拉|兰比尔·卡普尔|伊利亚娜·狄克鲁兹|Akash Khurana|沙鲁巴·舒克拉,印度,阿努拉格·巴苏,感人|经典|励志|搞笑|文艺,8.3,1
4,10428476,小屁孩日记3,2018-02-07 20:47:00,剧情|喜剧,扎克瑞·戈登|瑞秋·哈里斯|戴文·博斯蒂克|罗伯特·卡普荣|佩顿·李斯特,加拿大,大卫·博维斯,青春|搞笑,6.7,1
5,10430817,隐墙,2018-02-07 21:03:06,剧情|奇幻|悬疑|科幻,马蒂娜·格德克|Karlheinz Hackl|Wolfgang M|Bauer|Ulrik...,德国,Julian Pölsler,文艺,7.4,1
6,10432911,流感,2018-02-07 20:28:33,惊悚|爱情|剧情|悬疑|灾难|动作,张赫|秀爱|朴敏荷|柳海真|李熙俊,韩国,金成洙,感人,7.8,1
7,10437779,新世界,2018-02-07 20:24:56,惊悚|爱情|剧情|犯罪|悬疑|传记|历史|动作,李政宰|崔岷植|黄政民|宋智孝|朴圣雄,韩国,朴勋政,经典|黑帮|文艺,8.6,1
8,10437865,青春感恩记《父亲》之《父子篇》,2018-02-07 20:51:58,剧情,肖央|吴坚|李佳蔚,中国大陆,肖央,感人,6.3,1
9,10438426,浮城谜事,2018-02-07 20:26:53,剧情|犯罪|悬疑|爱情,郝蕾|秦昊|齐溪|朱亚文|祖锋,法国,娄烨,女性|文艺,7.3,1


In [0]:
def get_dim_dict(df, dim_name):
  type_list = list(map(lambda x:x.split('|') ,df[dim_name]))
  type_list = [x for l in type_list for x in l]
  type_list = sorted(list(set(type_list)))
  type_zip = zip(list(range(len(type_list))), type_list)
  type_dict = {}
  for i in type_zip:
    type_dict[i[1]] = i[0]
  return type_dict

In [0]:
type_dict = get_dim_dict(df, 'type')
actors_dict = get_dim_dict(df, 'actors')
director_dict = get_dim_dict(df, 'director')
trait_dict = get_dim_dict(df, 'trait')

In [105]:
print(type_dict)
print(actors_dict)
print(director_dict)
print(trait_dict)

{'传记': 0, '冒险': 1, '剧情': 2, '动作': 3, '历史': 4, '同性': 5, '喜剧': 6, '奇幻': 7, '恐怖': 8, '悬疑': 9, '情色': 10, '惊悚': 11, '战争': 12, '歌舞': 13, '武侠': 14, '灾难': 15, '爱情': 16, '犯罪': 17, '科幻': 18, '西部': 19, '音乐': 20}
{'50分': 0, 'AJ·鲍恩': 1, 'Aaron Grady Shaw': 2, 'Abdelkrim Bahloul': 3, 'Adam Butcher': 4, 'Adam Savage': 5, 'Adamo Ruggiero': 6, 'Adele Heather Taylor': 7, 'Adolf Hitler': 8, 'Adrianne Palicki': 9, 'Adélaïde Leroux': 10, 'Agame Malembo-Emene': 11, 'Aiden Lithgow': 12, 'Ainsworth': 13, 'Akash Khurana': 14, 'Aldevina Da Silva': 15, 'Aldri Anunciação': 16, 'Alena Vránová': 17, 'Alessandro Gassman': 18, 'Alev Uçarer': 19, 'Alex Kendrick': 20, 'Alex Shinohara': 21, 'Alexandra Neldel': 22, 'Alexandra Staden': 23, 'Alexandre Belin': 24, 'Alexis Ayala': 25, 'Alexis Bledel': 26, 'Alexis Knapp': 27, 'Alexis Zegerman': 28, 'Alicia Mezza': 29, 'Alicia Witt': 30, 'Alicja Bachleda-Curus': 31, 'Aliki Georgouli': 32, 'Allen Grigorian': 33, 'Allen Leong': 34, 'Allison Mackie': 35, 'Allison McAtee': 36, 'Al

In [0]:
df_ = df.drop(['ADD_TIME', 'enable', 'rat', 'id', 'name'], axis=1)

In [126]:
movie_dict_list = []
for i in df_.head(5).index:
  movie_dict = {}
  #type
  for s_type in df_.iloc[i]['type'].split('|'):
    movie_dict[s_type] = 1
  #actors
  
    
  

剧情|冒险|科幻|爱情
奇幻|惊悚|爱情|战争|剧情|科幻|冒险|动作
喜剧|犯罪|惊悚|动作
剧情|喜剧|音乐|爱情
剧情|喜剧


In [0]:
_m = np.random.randn(5,7)

In [0]:
from sklearn.metrics.pairwise import pairwise_distances
item_similarity = pairwise_distances(_m, metric='cosine')

In [27]:
_m

array([[ 1.873,  0.952, -0.311,  0.598, -0.361, -0.541, -0.712],
       [-0.664,  1.341, -1.274, -0.111, -0.011, -0.676, -0.773],
       [-0.249,  0.851, -1.435, -1.191,  0.111,  0.238, -0.908],
       [-0.498, -0.203,  0.442,  0.129,  0.649,  0.768,  1.14 ],
       [ 0.919,  1.038,  0.282, -0.638, -0.811, -0.676, -1.242]])

In [28]:
item_similarity

array([[0.   , 0.76 , 0.898, 1.656, 0.302],
       [0.76 , 0.   , 0.245, 1.518, 0.616],
       [0.898, 0.245, 0.   , 1.425, 0.631],
       [1.656, 1.518, 1.425, 0.   , 1.818],
       [0.302, 0.616, 0.631, 1.818, 0.   ]])